# 2019 OK JPL Horizon vs OpenOrb comparison

## Assumptions/ accountability-sourced documentation:

* Values for chosen orbit found here: https://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2019%20OK
* Location: I11, Gemini South
* Time: 2010/01/01 to 2020/01/01, midnights

## Current questions/ issues:

* Do I need to cite things in-code? nah, but track examples/ code snippets
* Do we need a maximum difference function or class? function

In [1]:
%matplotlib inline

import sys
import numpy as np
import astropy as ap
import astropy.units as u
from astropy.table import QTable
import matplotlib.pyplot as plt
import astroquery as aq
from astroquery.jplhorizons import Horizons

In [2]:
# "Global" variables: times, location, object
start_time = '2010-01-01T00:00:00'
stop_time = '2020-01-01T00:00:00'
element_time = ap.time.Time(start_time).jd
obj_id = '2019 OK'
Gemini_S = 'I11@399'

In [3]:
import pyoorb as oo
oo.pyoorb.oorb_init()

0

In [4]:
el_obj = Horizons(id=obj_id, location= '500@399',
               epochs=element_time)
el = el_obj.elements()
print(el.columns)

<TableColumns names=('targetname','datetime_jd','datetime_str','H','G','e','q','incl','Omega','w','Tp_jd','n','M','nu','a','Q','P')>


In [7]:
'''
Gfortran version 4.8.6

Useful bits from docs:


pyoorb uses the same input orbit format in all functions. Orbits have to be in the form of a numpy.array 
(two-dimensional for multiple orbits). Every orbit array has to contain 12 properties (in this order), some of which 
depend on the type of the elements provided (COM for cometary orbits):


JPL: 'targetname','datetime_jd','datetime_str','H','G','e','q','incl','Omega','w','Tp_jd','n','M','nu','a','Q','P'

orbit id: an integer number to identify the orbit; usually ranges from 0 to n-1, where n is the number of orbits
perihelion distance (au) for COM, semimajor axis a (au) for KEP, x (au) for CART
eccentricity for COM or KEP, y (au) for CART
inclination (deg) for COM or KEP, z (au) for CART
longitude of the ascending node (deg) for COM and KEP, dx/dt (au/day) for CART
argument of the periapsis (deg) for COM and KEP, dy/dt (au/day) for CARqT
epoch of perihelion (modified Julian date) for COM, mean anomaly (deg) for KEP, dz/dt for CART
orbital element type; integer value: CART: 1, COM: 2, KEP: 3
epoch of the osculating elements (modified Julian date)
timescale type of the epochs provided; integer value: UTC: 1, UT1: 2, TT: 3, TAI: 4
absolute magnitude of the target
photometric slope parameter of the target

pyoorb.oorb_ephemeris_basic provides these properties, enabling fast computations and requiring less memory:

modified julian date
right ascension (deg)
declination (deg)
dra/dt sky-motion (deg/day, including cos(dec) factor)
ddec/dt sky-motion (deg/day)
solar phase angle (deg)
solar elongation angle (deg)
heliocentric distance (au)
geocentric distance (au)
predicted apparent V-band magnitude
true anomaly (deg)


JPL TableColumns names=
'targetname',
'datetime_str',
'datetime_jd',
'H',
'G',
'solar_presence',
'flags',
'RA',
'DEC',
'RA_app',
'DEC_app',
'RA_rate',
'DEC_rate',
'AZ','EL',
'AZ_rate',
'EL_rate',
'sat_X',
'sat_Y',
'sat_PANG',
'siderealtime',
'airmass',
'magextinct',
'V',
'surfbright',
'illumination',
'illum_defect',
'sat_sep',
'sat_vis',
'ang_width',
'PDObsLon',
'PDObsLat',
'PDSunLon',
'PDSunLat',
'SubSol_ang',
'SubSol_dist',
'NPole_ang',
'NPole_dist',
'EclLon',
'EclLat',
'r',
'r_rate',
'delta',
'delta_rate',
'lighttime',
'vel_sun',
'vel_obs',
'elong',
'elongFlag',
'alpha',
'lunar_elong',
'lunar_illum',
'sat_alpha',
'sunTargetPA',
'velocityPA',
'OrbPlaneAng',
'constellation',
'TDB-UT',
'ObsEclLon',
'ObsEclLat',
'NPole_RA',
'NPole_DEC',
'GlxLon',
'GlxLat',
'solartime',
'earth_lighttime',
'RA_3sigma',
'DEC_3sigma',
'SMAA_3sigma',
'SMIA_3sigma',
'Theta_3sigma',
'Area_3sigma',
'RSS_3sigma',
'r_3sigma',
'r_rate_3sigma',
'SBand_3sigma',
'XBand_3sigma',
'DoppDelay_3sigma',
'true_anom',
'hour_angle',
'alpha_true',
'PABLon',
'PABLat')>


make ephem
oorb --task=propagation --epoch-mjd-utc=DATE --orb-in=ORBITFILEIN --orb-out=ORBITFILEOUT
oorb --task=ephemeris --orb-in=ORBBITFILEOUT
pyoorb.oorb_ephemeris_basic over pyoorb.oorb_ephemeris_full
Below is an example copied from--> https://github.com/oorb/oorb/tree/master/python#ephemeris-computation
It felt useful to study, I intend to shuffle things around to get to what I need from it
'''

"\nGfortran version 4.8.6\n\nUseful bits from docs:\n\n\npyoorb uses the same input orbit format in all functions. Orbits have to be in the form of a numpy.array \n(two-dimensional for multiple orbits). Every orbit array has to contain 12 properties (in this order), some of which \ndepend on the type of the elements provided (COM for cometary orbits):\n\n\nJPL: 'targetname','datetime_jd','datetime_str','H','G','e','q','incl','Omega','w','Tp_jd','n','M','nu','a','Q','P'\n\norbit id: an integer number to identify the orbit; usually ranges from 0 to n-1, where n is the number of orbits\nperihelion distance (au) for COM, semimajor axis a (au) for KEP, x (au) for CART\neccentricity for COM or KEP, y (au) for CART\ninclination (deg) for COM or KEP, z (au) for CART\nlongitude of the ascending node (deg) for COM and KEP, dx/dt (au/day) for CART\nargument of the periapsis (deg) for COM and KEP, dy/dt (au/day) for CARqT\nepoch of perihelion (modified Julian date) for COM, mean anomaly (deg) for 

In [5]:
# orbits is where selected element values go that we got from JPL
orbits = np.array([[0, 1.46905, 0.33435, np.deg2rad(14.3024),
                    np.deg2rad(224.513), np.deg2rad(27.5419),
                    np.deg2rad(324.697), 3, 51544.5, 1, 12.5, 0.15]],
                    dtype=np.double, order='F')

epoch = np.array([51232.23, 3], dtype=np.double, order='F')
orb, err = oo.pyoorb.oorb_propagation(in_orbits=orbits,
                                     in_epoch=epoch,
                                    in_dynmodel='N')
                                    
mjds = np.arange(51232, 51233, 1/24)
epochs = np.array(list(zip(mjds, [1]*len(mjds))), dtype=np.double, order='F')
eph, err = oo.pyoorb.oorb_ephemeris_full(in_orbits=orbits,
                                         in_obscode='568',
                                         in_date_ephems=epochs,
                                         in_dynmodel='N')
print(err, eph)

0 [[[ 5.12320000e+04  2.97420305e+01  9.39898382e+00  4.54411204e-01
    1.15967356e-01  2.55335063e+01  5.70687690e+01  1.92629817e+00
    2.27595666e+00  1.68784924e+01  7.56834978e+01  3.09658224e+01
   -2.57079287e+00 -1.22968568e+02 -2.57187230e+00  5.65262567e+01
   -3.03728791e+00 -9.74081340e+01 -3.03836734e+00  5.67373413e+01
    5.36581121e+01  2.79270091e+01  4.93916942e-01  3.23198414e+01
    1.06096650e+00  1.60454266e+00 -1.02066546e-01 -7.60331277e-03
    6.44843751e-03 -2.53113453e-03 -8.88643901e-01  4.34680947e-01
    1.86372479e-05  1.64854703e+02]
  [ 5.12320417e+04  2.97612057e+01  9.40381070e+00  4.53732760e-01
    1.15720650e-01  2.55263237e+01  5.70468341e+01  1.92635305e+00
    2.27643567e+00  1.68787982e+01  7.56922196e+01  3.09852619e+01
   -2.57280325e+00 -1.22990606e+02 -2.57364094e+00  5.65385408e+01
   -3.04034223e+00 -9.74373272e+01 -3.04117992e+00  7.00074489e+01
    4.35947788e+01  4.16228206e+01  4.97711138e-01  3.27693280e+01
    1.06064968e+00  1.60

In [6]:
# "Global" variables: times, location, object
start_time = '2010-01-01T00:00:00'
stop_time = '2020-01-01T00:00:00'
internal_start = ap.time.Time(start_time)
internal_stop = ap.time.Time(stop_time)
element_time = internal_start.jd
obj_id = '2019 OK'
Gemini_S = 'I11@399'

In [7]:
# time conversions, epochs
element_time_pyoorb = internal_start.mjd
start_pyoorb = internal_start.mjd
stop_pyoorb = internal_stop.mjd
peri_time = ap.time.Time(el['Tp_jd'][0], format='jd').mjd

In [8]:
orbits = np.array([[0, 1.46905, 0.33435, np.deg2rad(14.3024),
                    np.deg2rad(224.513), np.deg2rad(27.5419),
                    np.deg2rad(324.697), 3, 51544.5, 1, 12.5, 0.15]],
                    dtype=np.double, order='F')
reorg = np.array([[1, el['q'][0], el['e'][0], np.deg2rad(el['incl'][0]),
                   np.deg2rad(el['Omega'][0]), np.deg2rad(el['w'][0]),
                   peri_time, 2, start_pyoorb, 1, el['H'][0], el['G'][0]]],
                   dtype=np.double, order='F')

epoch = np.array([element_time_pyoorb, 1], dtype=np.double, order='F')

orb0, err0 = oo.pyoorb.oorb_propagation(in_orbits=orbits,
                                     in_epoch=epoch,
                                    in_dynmodel='N') # change backto N model later

orb1, err1 = oo.pyoorb.oorb_propagation(in_orbits=reorg,
                                     in_epoch=epoch,
                                    in_dynmodel='N')

In [ ]:
mjds = np.arange(start_pyoorb, stop_pyoorb, 1)
epochs = np.array(list(zip(mjds, [1]*len(mjds))), dtype=np.double, order='F')
eph, err = oo.pyoorb.oorb_ephemeris_basic(in_orbits=reorg,
                                         in_obscode='568',
                                         in_date_ephems=epochs,
                                         in_dynmodel='2')
print(err, eph)

In [ ]:
# orbits is where selected element values go that we got from JPL


mjds = np.arange(start_pyoorb, stop_pyoorb, 1)

epochs = np.array(list(zip(mjds, [1]*len(mjds))), dtype=np.double, order='F')


eph, err = oo.pyoorb.oorb_ephemeris_basic(in_orbits=reorg,
                                         in_obscode='568',
                                         in_date_ephems=epochs,
                                         in_dynmodel='2')

print(eph, err)

# jpl to oorb function

# "Final" Draft Things Below This Line

In [5]:
# "Global" variables: times, location, object
start_time = '2010-01-01T00:00:00'
stop_time = '2020-01-01T00:00:00'
element_time = ap.time.Time(start_time).jd
obj_id = '2019 OK'
Gemini_S = 'I11@399'

## JPL-Sourced Values

In [6]:
# 2019 OK elements. NB: location is at center of Earth bc elements must be object-centered
el_obj = Horizons(id=obj_id, location= '500@399',
               epochs=element_time)
el = el_obj.elements()
print(el.columns)

<TableColumns names=('targetname','datetime_jd','datetime_str','H','G','e','q','incl','Omega','w','Tp_jd','n','M','nu','a','Q','P')>


In [9]:
# 2019 OK ephem
ephem_obj = Horizons(id= obj_id, location= Gemini_S,
               epochs={'start': start_time, 'stop':stop_time,
                      'step':'1d'})
eph = ephem_obj.ephemerides()
print(eph.columns)

<TableColumns names=('targetname','datetime_str','datetime_jd','H','G','solar_presence','flags','RA','DEC','RA_app','DEC_app','RA_rate','DEC_rate','AZ','EL','AZ_rate','EL_rate','sat_X','sat_Y','sat_PANG','siderealtime','airmass','magextinct','V','surfbright','illumination','illum_defect','sat_sep','sat_vis','ang_width','PDObsLon','PDObsLat','PDSunLon','PDSunLat','SubSol_ang','SubSol_dist','NPole_ang','NPole_dist','EclLon','EclLat','r','r_rate','delta','delta_rate','lighttime','vel_sun','vel_obs','elong','elongFlag','alpha','lunar_elong','lunar_illum','sat_alpha','sunTargetPA','velocityPA','OrbPlaneAng','constellation','TDB-UT','ObsEclLon','ObsEclLat','NPole_RA','NPole_DEC','GlxLon','GlxLat','solartime','earth_lighttime','RA_3sigma','DEC_3sigma','SMAA_3sigma','SMIA_3sigma','Theta_3sigma','Area_3sigma','RSS_3sigma','r_3sigma','r_rate_3sigma','SBand_3sigma','XBand_3sigma','DoppDelay_3sigma','true_anom','hour_angle','alpha_true','PABLon','PABLat')>


## OpenOrb-Sourced Values

In [ ]:
# elements reorganized


# COM used!


reorg = np.array([[1, el['q'][0], el['e'][0], np.deg2rad(el['incl'][0]), 
                  np.deg2rad(el['Omega'][0]),#asc node?
                  np.deg2rad(el['w'][0]), el['Tp_jd'][0], #make mod JD
                  2, 1, el['M'][0], el['H'][0]]],
                 dtype=np.double, order='F')



start_orb = ap.time.Time(start_time).mjd
end_orb = ap.time.Time(stop_time).mjd


epoch = np.array([51232.23, 3], dtype=np.double, order='F')
orb, err = oo.pyoorb.oorb_propagation(in_orbits=example,
                                     in_epoch=epoch,
                                    in_dynmodel='2')
                                 
mjds = np.arange(start_orb, end_orb, 1/3652)
epochs = np.array(list(zip(mjds, [1]*len(mjds))), dtype=np.double, order='F')
eph, err = oo.pyoorb.oorb_ephemeris_basic(in_orbits=example,
                                         in_obscode='568',
                                         in_date_ephems=epochs,
                                         in_dynmodel='2')
print(err, eph)                       
                   

'targetname','datetime_jd','datetime_str','H','G','e','q','incl','Omega','w','Tp_jd','n','M','nu','a','Q','P'
'''
yes orbit id: an integer number to identify the orbit; usually ranges from 0 to n-1, where n is the number of orbits
yes perihelion distance (au) for COM
yes eccentricity for COM
yes inclination (deg) for COM
yes? longitude of the ascending node (deg) for COM
yes? argument of the periapsis (deg) for COM
yes epoch of perihelion (modified Julian date) for COM
yes orbital element type; integer value: COM: 2
yes? epoch of the osculating elements (modified Julian date)
yes timescale type of the epochs provided; integer value: UTC: 1, UT1: 2, TT: 3, TAI: 4
yes? absolute magnitude of the target
yes? photometric slope parameter of the target
'''
# jpl to oorb function

## Difference metric

In [ ]:
# difference function

# use great circle difference, keep coordinates and magnitude

def max_diff(el, ephem1, ephem2):
    '''
    el: elements of orbit, numpy array
    ephem1: elements of ephemerides, 1st integrator, numpy array?
    ephem2: elements of ephemerides, 2nd integrator, numpy array? (same type as ephem1)
    
    '''
    # don't forget to keep the 360deg problem in mind
    
    return diff_matrix